In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from tqdm import tqdm
import numpy as np
import random

# ============================
# 1. Setup and Hyperparameters
# ============================

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
num_epochs = 30
batch_size = 4096  # Large batch size
learning_rate = 0.1
momentum = 0.9
weight_decay = 5e-4

# ============================
# 2. Data Loading and Preprocessing
# ============================

# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

# Only normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 training and test datasets
print("Loading CIFAR-10 dataset...")
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

# Convert datasets to tensors and move to GPU
print("Converting training data to tensors and moving to GPU...")
train_inputs = torch.stack([trainset[i][0] for i in range(len(trainset))]).to(device)
train_targets = torch.tensor([trainset[i][1] for i in range(len(trainset))]).to(device)

print("Converting test data to tensors and moving to GPU...")
test_inputs = torch.stack([testset[i][0] for i in range(len(testset))]).to(device)
test_targets = torch.tensor([testset[i][1] for i in range(len(testset))]).to(device)

print("Data loading complete.")

# ============================
# 3. Model Definition
# ============================

class ResNet18_CIFAR10(nn.Module):
    def __init__(self):
        super(ResNet18_CIFAR10, self).__init__()
        self.model = resnet18(pretrained=False, num_classes=10)
        # Modify the first convolution layer for CIFAR-10
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()

    def forward(self, x):
        return self.model(x)

model = ResNet18_CIFAR10().to(device)

# ============================
# 4. Optimizer and Scheduler
# ============================

optimizer = optim.SGD(model.parameters(), lr=learning_rate, 
                      momentum=momentum, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 25], gamma=0.1)

# Define loss function
criterion = nn.CrossEntropyLoss()

# ============================
# 5. Training and Evaluation Functions
# ============================

def train_baseline(model, train_inputs, train_targets, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Shuffle the data at the beginning of each epoch
    perm = torch.randperm(train_inputs.size(0))
    shuffled_inputs = train_inputs[perm]
    shuffled_targets = train_targets[perm]

    num_batches = train_inputs.size(0) // batch_size

    loop = tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch_idx in loop:
        # Create batch
        start = batch_idx * batch_size
        end = start + batch_size
        inputs = shuffled_inputs[start:end]
        targets = shuffled_targets[start:end]

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # Update progress bar
        loop.set_postfix(loss=running_loss/(batch_idx+1), acc=100.*correct/total)

def evaluate_baseline(model, test_inputs, test_targets):
    model.eval()
    correct_clean = 0
    total_clean = 0

    with torch.no_grad():
        # Clean accuracy
        outputs = model(test_inputs)
        _, predicted = outputs.max(1)
        total_clean += test_targets.size(0)
        correct_clean += predicted.eq(test_targets).sum().item()

    clean_acc = 100. * correct_clean / total_clean

    print(f"\nClean Accuracy: {clean_acc:.2f}%\n")

    return clean_acc

# ============================
# 6. Training Loop
# ============================

best_clean_acc = 0.0

for epoch in range(num_epochs):
    train_baseline(model, train_inputs, train_targets, optimizer, epoch)
    clean_acc = evaluate_baseline(model, test_inputs, test_targets)
    scheduler.step()

    # Save the best model based on clean accuracy
    if clean_acc > best_clean_acc:
        best_clean_acc = clean_acc
        torch.save(model.state_dict(), "resnet18_cifar10_baseline_best.pth")
        print(f"Best model saved with clean accuracy: {best_clean_acc:.2f}%")

print("Training complete.")
print(f"Best Clean Accuracy: {best_clean_acc:.2f}%")


Using device: cuda
Loading CIFAR-10 dataset...
Files already downloaded and verified
Files already downloaded and verified
Converting training data to tensors and moving to GPU...
Converting test data to tensors and moving to GPU...
Data loading complete.


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/30: 100%|██████████| 12/12 [00:07<00:00,  1.51it/s, acc=14.9, loss=3.36]



Clean Accuracy: 10.00%

Best model saved with clean accuracy: 10.00%


Epoch 2/30: 100%|██████████| 12/12 [00:07<00:00,  1.55it/s, acc=16.9, loss=2.41]



Clean Accuracy: 11.73%

Best model saved with clean accuracy: 11.73%


...


Clean Accuracy: 71.26%

Best model saved with clean accuracy: 71.26%


Epoch 29/30: 100%|██████████| 12/12 [00:07<00:00,  1.55it/s, acc=86.9, loss=0.409]



Clean Accuracy: 71.11%



Epoch 30/30: 100%|██████████| 12/12 [00:07<00:00,  1.55it/s, acc=87.1, loss=0.406]



Clean Accuracy: 71.21%

Training complete.
Best Clean Accuracy: 71.26%


In [5]:
def pgd_attack(model, images, labels, eps=8/255, alpha=2/255, iters=7):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    ori_images = images.clone().detach()

    for i in range(iters):
        # Set requires_grad to True for gradient computation
        images.requires_grad = True

        outputs = model(images)
        loss = criterion(outputs, labels)
        model.zero_grad()
        loss.backward()

        # Collect the gradients
        grad = images.grad.data

        # Update adversarial images
        images = images + alpha * torch.sign(grad)

        # Clamp perturbations
        perturbation = torch.clamp(images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + perturbation, min=0, max=1).detach()

    return images


# ============================
# 7. Evaluating Baseline on Adversarial Examples
# ============================

def evaluate_baseline_on_adversarial(model, test_inputs, test_targets, eps=8/255, alpha=2/255, pgd_steps=7):
    model.eval()
    correct_adv = 0
    total_adv = 0

    # Loop through test data in batches
    for i in tqdm(range(0, test_inputs.size(0), batch_size), desc="Evaluating Adversarial Accuracy"):
        start = i
        end = min(i + batch_size, test_inputs.size(0))
        inputs = test_inputs[start:end]
        targets = test_targets[start:end]

        # Generate adversarial examples using PGD (requires gradients)
        adv_inputs = pgd_attack(model, inputs, targets, eps=eps, alpha=alpha, iters=pgd_steps)

        # Now evaluate the model on adversarial examples (no need for gradients here)
        with torch.no_grad():
            outputs = model(adv_inputs)
            _, predicted = outputs.max(1)
            total_adv += targets.size(0)
            correct_adv += predicted.eq(targets).sum().item()

    # Calculate adversarial accuracy
    adv_acc = 100. * correct_adv / total_adv
    print(f"\nAdversarial Accuracy on Baseline Model: {adv_acc:.2f}%\n")

    return adv_acc


# After the baseline model has been trained, test its robustness on adversarial inputs
adv_acc_baseline = evaluate_baseline_on_adversarial(model, test_inputs, test_targets)


Evaluating Adversarial Accuracy: 100%|██████████| 3/3 [00:10<00:00,  3.65s/it]


Adversarial Accuracy on Baseline Model: 16.14%



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from tqdm import tqdm
import numpy as np
import random

# ============================
# 1. Setup and Hyperparameters
# ============================

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
num_epochs = 30
batch_size = 4096  # Large batch size
learning_rate = 0.1
momentum = 0.9
weight_decay = 5e-4
pgd_steps = 7
pgd_alpha = 2/255
pgd_eps = 8/255

# ============================
# 2. Data Loading and Preprocessing
# ============================

# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

# Only normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 training and test datasets
print("Loading CIFAR-10 dataset...")
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

# Convert datasets to tensors and move to GPU
print("Converting training data to tensors and moving to GPU...")
train_inputs = torch.stack([trainset[i][0] for i in range(len(trainset))]).to(device)
train_targets = torch.tensor([trainset[i][1] for i in range(len(trainset))]).to(device)

print("Converting test data to tensors and moving to GPU...")
test_inputs = torch.stack([testset[i][0] for i in range(len(testset))]).to(device)
test_targets = torch.tensor([testset[i][1] for i in range(len(testset))]).to(device)

print("Data loading complete.")

# ============================
# 3. Model Definition
# ============================

class ResNet18_CIFAR10(nn.Module):
    def __init__(self):
        super(ResNet18_CIFAR10, self).__init__()
        self.model = resnet18(pretrained=False, num_classes=10)
        # Modify the first convolution layer for CIFAR-10
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()

    def forward(self, x):
        return self.model(x)

model = ResNet18_CIFAR10().to(device)

# ============================
# 4. Optimizer and Scheduler
# ============================

optimizer = optim.SGD(model.parameters(), lr=learning_rate, 
                      momentum=momentum, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 25], gamma=0.1)

# Define loss function
criterion = nn.CrossEntropyLoss()

# ============================
# 5. PGD Attack Implementation
# ============================

def pgd_attack(model, images, labels, eps=8/255, alpha=2/255, iters=7):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    ori_images = images.clone().detach()

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)
        loss = criterion(outputs, labels)
        model.zero_grad()
        loss.backward()
        grad = images.grad.data

        # Update adversarial images
        images = images + alpha * torch.sign(grad)
        # Clamp perturbations
        perturbation = torch.clamp(images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + perturbation, min=0, max=1).detach()

    return images

# ============================
# 6. Training and Evaluation Functions
# ============================

def train_adversarial(model, train_inputs, train_targets, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Shuffle the data at the beginning of each epoch
    perm = torch.randperm(train_inputs.size(0))
    shuffled_inputs = train_inputs[perm]
    shuffled_targets = train_targets[perm]

    num_batches = train_inputs.size(0) // batch_size

    loop = tqdm(range(num_batches), desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch_idx in loop:
        # Create batch
        start = batch_idx * batch_size
        end = start + batch_size
        inputs = shuffled_inputs[start:end]
        targets = shuffled_targets[start:end]

        # Generate adversarial examples
        adv_inputs = pgd_attack(model, inputs, targets, eps=pgd_eps, alpha=pgd_alpha, iters=pgd_steps)

        # Combine clean and adversarial examples
        combined_inputs = torch.cat([inputs, adv_inputs], dim=0)
        combined_targets = torch.cat([targets, targets], dim=0)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(combined_inputs)
        loss = criterion(outputs, combined_targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = outputs.max(1)
        total += combined_targets.size(0)
        correct += predicted.eq(combined_targets).sum().item()

        # Update progress bar
        loop.set_postfix(loss=running_loss/(batch_idx+1), acc=100.*correct/total)

def evaluate_adversarial(model, test_inputs, test_targets):
    model.eval()
    correct_clean = 0
    total_clean = 0
    correct_adv = 0
    total_adv = 0

    with torch.no_grad():
        # Clean accuracy
        outputs = model(test_inputs)
        _, predicted = outputs.max(1)
        total_clean += test_targets.size(0)
        correct_clean += predicted.eq(test_targets).sum().item()

    # Evaluate adversarial accuracy
    for i in tqdm(range(0, test_inputs.size(0), batch_size), desc="Evaluating Adversarial"):
        start = i
        end = min(i + batch_size, test_inputs.size(0))
        inputs = test_inputs[start:end]
        targets = test_targets[start:end]

        # Generate adversarial examples
        adv_inputs = pgd_attack(model, inputs, targets, eps=pgd_eps, alpha=pgd_alpha, iters=pgd_steps)

        # Predict on adversarial examples
        outputs = model(adv_inputs)
        _, predicted = outputs.max(1)
        total_adv += targets.size(0)
        correct_adv += predicted.eq(targets).sum().item()

    clean_acc = 100. * correct_clean / total_clean
    adv_acc = 100. * correct_adv / total_adv

    print(f"\nClean Accuracy: {clean_acc:.2f}%")
    print(f"Adversarial Accuracy: {adv_acc:.2f}%\n")

    return clean_acc, adv_acc

# ============================
# 7. Training Loop
# ============================

best_adv_acc = 0.0

for epoch in range(num_epochs):
    train_adversarial(model, train_inputs, train_targets, optimizer, epoch)
    clean_acc, adv_acc = evaluate_adversarial(model, test_inputs, test_targets)
    scheduler.step()

    # Save the best model based on adversarial accuracy
    if adv_acc > best_adv_acc:
        best_adv_acc = adv_acc
        torch.save(model.state_dict(), "resnet18_cifar10_adversarial_best.pth")
        print(f"Best model saved with adversarial accuracy: {best_adv_acc:.2f}%")

print("Training complete.")
print(f"Best Adversarial Accuracy: {best_adv_acc:.2f}%")
